# Prepare PySM 'f2' free-free model for PySM

This notebook implements the logpoltens small-scale–injection workflow for free–free intensity maps (i.e., without polarization components).

The goal is to generate a high-resolution free–free template by injecting statistically consistent small-scale structure while preserving the large-scale morphology.

The input free–free tracer used here is the emission-measure (EM) mean/variance map from the all-sky Galactic Faraday/electron-density reconstruction of Hutschenreuter et al. 2024.

Throughout this notebook, we refer to this map as the HE map, following Hutschenreuter & Enßlin (2020), which presented the original reconstruction framework on which the later analysis is based.

We use the FITS product EM_mean_std.fits provided on Zenodo: https://zenodo.org/records/10523170/files/EM_mean_std.fits?download=1


The small-scale injection follows the same logpoltens formalism used in modern PySM3 foreground templates (adapted here to intensity-only free–free): https://arxiv.org/abs/2502.20452

In [ ]:
import os

# Prefer externally provided values; otherwise choose a sane default.
os.environ.setdefault(
    "OMP_NUM_THREADS",
    os.environ.get("SLURM_CPUS_PER_TASK") or str(os.cpu_count() or 1),
)
print("OMP_NUM_THREADS =", os.environ["OMP_NUM_THREADS"])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import pysm3
from pathlib import Path
import os
import pysm3.units as u
import pymaster as nmt
from scipy.optimize import curve_fit


In [ ]:
import os
from pathlib import Path
from datetime import date
import urllib.request

# Threading controlled via OMP_NUM_THREADS (set at top of notebook)

# Version tag for filenames / logs
today = date.today()
version = today.strftime("%Y.%m.%d")

# Track outputs produced by this notebook
output_files = []

# ---- Paths (relative to notebook location) ----
BASE_DIR = Path("")          # assuming notebook is in notebooks/
DATA_DIR = BASE_DIR / "ff_pysm_data"
# DATA_IN  = DATA_DIR / "input"
# DATA_OUT = DATA_DIR / "output"

DATA_DIR.mkdir(parents=True, exist_ok=True)
# DATA_OUT.mkdir(parents=True, exist_ok=True)

version

##### Specify the nside and lmax for the output map here

In [ ]:
nside_final=2048
lmax_final=3*nside_final-1


## Reading the HE free-free EM map, Processing

In [ ]:
url = "https://zenodo.org/records/10523170/files/EM_mean_std.fits?download=1"
imapfile = DATA_DIR / "HE_EM_mean_std.fits"

if not imapfile.exists():
    print(f"Downloading input map to {imapfile} ...")
    urllib.request.urlretrieve(url, imapfile)
else:
    print(f"Using existing file {imapfile}")

In [ ]:
HE_EM=hp.read_map(imapfile) #EM mean is the first column, cm^-6 pc

In [ ]:
hp.mollview(HE_EM, title='HE EM Map', unit=r'${\rm cm}^{-6} {\rm pc}$', cmap='turbo',norm='hist')
hp.graticule(dpar=30)

The sharp horizontal boundary visible in the northern high-latitude region is inherited from the original Hα map used to construct this EM map, and results from the stitching of two independent Hα surveys.

In [ ]:
nside=hp.get_nside(HE_EM)
lmax=3*nside-1
print('The above map is at an nside and lmax of '+str(nside)+' and '+str(lmax)+', respectively.')

### Converting the Emission Measure map to Intensity in brightness temperature units (uK_RJ)

In [ ]:
#this function converts the EM map to a T_uRJ map

def convert_EM_to_TRJ(EMmap,freq,TeMap=7000):   #we use 7,000 K as the average value of the Electron temperature in the ISM (Dickinson et al 2003, Planck 2015 X)

    gff=np.log((np.exp(5.960-((np.sqrt(3)/np.pi)*np.log(freq*((TeMap*1e-4)**(-3/2))))))+np.exp(1))

    tau=0.05468*(TeMap**(-3/2))*EMmap*(freq**(-2))*gff

    TRJmap=1e6*TeMap*(1-np.exp(-tau))

    return TRJmap


In [ ]:
HE_ff_T=convert_EM_to_TRJ(HE_EM,freq=30.0)

In [ ]:
np.min(HE_ff_T),np.max(HE_ff_T)

In [ ]:
%matplotlib inline
hp.mollview(HE_ff_T,cmap='turbo', title='HE Free-free Intensity Map at 30 GHz', unit=r'${\rm T\,[\mu K}_{\rm RJ}]$',norm='hist')
hp.graticule(dpar=30)
#print(np.min(HE_ff_T),np.max(HE_ff_T))
#plt.savefig('HE_ff_T_30GHz.png', dpi=300)

### Conversion to LogMap

For intensity-only free-free (linear space) $I$, we define the log-space field $i$ as

$$
i = \ln(\max(I, \epsilon))
$$

where $\epsilon$ is a small positive floor to avoid $\ln(0)$.

The inverse transform is

$$
I = e^{i}.
$$


Note than Imap (in text) refers to the linear space map while imap (with small case) refers to the logspace map.

In [ ]:
def choose_eps(m, abs_floor=1e-30, frac_min=0.1, frac_median=1e-12):
    # Choose a log-floor eps from the data.
    # Intended to only affect true zeros/underflows while being robust to outliers.
    m = np.asarray(m, dtype=float)
    pos = m[np.isfinite(m) & (m > 0)]
    if pos.size == 0:
        return abs_floor
    return max(abs_floor, frac_min * float(pos.min()), frac_median * float(np.median(pos)))


def map_to_log_pol_tens(m, eps):
    m = np.asarray(m, dtype=float)
    m = np.maximum(m, eps)  # avoid log(0) -> -inf
    return np.log(m)


def log_pol_tens_to_map(log_pol_tens):
    log_pol_tens = np.asarray(log_pol_tens, dtype=float)
    return np.exp(log_pol_tens)


In [ ]:
eps = choose_eps(HE_ff_T)
n_pix = HE_ff_T.size
n_zero = int(np.sum(HE_ff_T == 0))
n_neg = int(np.sum(HE_ff_T < 0))
n_nonfinite = int(np.sum(~np.isfinite(HE_ff_T)))
n_pos_below_eps = int(np.sum((HE_ff_T > 0) & (HE_ff_T < eps)))

print(f"eps = {eps:g} uK_RJ")
print(f"pixels: total={n_pix}, zero={n_zero}, neg={n_neg}, nonfinite={n_nonfinite}, 0<val<eps={n_pos_below_eps}")


In [ ]:
log_he_ff = map_to_log_pol_tens(HE_ff_T, eps)

In [ ]:
plt.figure(figsize=(12, 8))  # make the canvas big enough

hp.mollview(HE_ff_T,
        cmap="turbo",
        norm="hist",
        title='Original Free-Free Map (I)',
        sub=(1, 2, 1),  # (nrows, ncols, index)
    )

hp.mollview(log_he_ff,
        cmap="turbo",
        norm="hist",
        title='Logarithmic Free-Free Map (i)',
        sub=(1, 2, 2),  # (nrows, ncols, index)
    )

## Compute the anafast power spectrum

In [ ]:
def run_anafast(m, lmax):
    clanaf = hp.anafast(m, lmax=lmax,use_pixel_weights=True)
    cl = clanaf
    ell = np.arange(lmax + 1, dtype=float)

    cl_norm = ((ell * (ell + 1)) / (2*np.pi)) 
    cl_norm[0] = 1
    dl = ((ell * (ell + 1)) / (2*np.pi))*cl 
    dl[0] = 1

    return ell, cl_norm, cl,dl

In [ ]:
ell_HE,cl_norm_HE,cl_HE,dl_HE = run_anafast(log_he_ff, lmax)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

# First subplot: f1
plt.subplot(1, 2, 1)
plt.plot(ell_HE, cl_HE, color='darkorange')
plt.xscale('log')
plt.yscale('log')
#plt.xlim([2, lmax])
plt.grid()
plt.title(r"TT spectrum ($C_\ell$) for HE Free-free Intensity (imap)")
plt.xlabel(r"$\ell$")
plt.ylabel(r"$C_\ell\;$")

#\mu K}_{\rm RJ}
# Second subplot: f2 (replace with actual variables if you have them)
plt.subplot(1, 2, 2)
plt.plot(ell_HE, dl_HE, color='darkblue')  # Example second field
plt.xscale('log')
plt.yscale('log')
plt.xlim([2, lmax])
plt.grid()
plt.title(r"TT spectrum ($\mathcal{D}_\ell)$ for HE Free-free Intensity (imap)")
plt.xlabel(r"$\ell$")
plt.ylabel(r"$\ell(\ell+1)C_\ell/2\pi\;$")

plt.tight_layout()
plt.show()

## Fitting the Dl

#### Defining the Dl power law

In [ ]:
def powerlaw_model(ell, A, gamma):
    out = A * ell**gamma
    return out

#### Fitting with power-law

$$
C_{\ell}^{\rm ff}=A_{\rm ff}\ell^{\gamma}
$$

In [ ]:
ell_fit_low = 30
ell_fit_high = 150
gamma_fit2 = -1.5
A_fit, gamma_fit, A_fit_std, gamma_fit_std = {}, {}, {}, {}
A_fit2 = {}
smallscales = []
ell_pivot = 200

plt.figure(figsize=(10, 5))

xdata = np.arange(ell_fit_low, ell_fit_high)
ydata = xdata * (xdata + 1) / np.pi / 2 * cl_HE[xdata]
(A_fit, gamma_fit), cov = curve_fit(powerlaw_model, xdata, ydata)
A_fit2 = np.fabs(A_fit) * ell_fit_high ** (
    gamma_fit - gamma_fit2
)



plt.loglog(ell_HE, (ell_HE * (ell_HE + 1) / (2*np.pi) ) * cl_HE,color='darkblue')

plt.xlim([2,3*lmax])
scaling = np.zeros_like(ell_HE)
scaling[2:] = powerlaw_model(ell_HE[2:], A_fit2, gamma_fit2)
scaling[:2] = 0
plt.plot(ell_HE[2:], scaling[2:], label=r"$\gamma$" + f"={gamma_fit2}",color='tomato',ls='--')
#smallscales.append(scaling)

print(A_fit, gamma_fit)
print(A_fit2, gamma_fit2)

plt.axvline(ell_fit_low, linestyle=":", color="gray",
            label=f"$\\ell = {ell_fit_low}$")
plt.axvline(ell_fit_high, linestyle=":", color="gray",
            label=f"$\\ell = {ell_fit_high}$")
plt.axvline(x=180, linestyle="--", color="gray",
            label=r"$\ell = 180$")

#plt.axvline(ell_pivot, linestyle="--", color="k")
#plt.grid()
plt.title("TT spectrum for HE Free-free Intensity",fontsize=15)

plt.xlabel((r"$\ell$"),fontsize=13.5)
#plt.legend(fontsize=15)

plt.legend()
plt.ylabel(r"$\ell(\ell+1)C_\ell / 2\pi\;$", fontsize=13);

#plt.savefig("HE_freefree_Dl.png", dpi=300)


# Generating the small-scale realizations

##### Defining and visualizing the sigmoid function

We use the sigmoid function to isolate or emphasize specific scales of interest in our analysis. It is defined as,

$$
\frac{1}{1 + \exp\left( -\frac{\text{power} \cdot \left(x - x_0 - \frac{\text{width}}{2}\right)}{\text{width}} \right)}
$$

In [ ]:
def sigmoid(x, x0, width, power=4):
    """Sigmoid function given start point and width
    Parameters
    ----------
    x : array
        input x axis
    x0 : float
        value of x where the sigmoid starts (not the center)
    width : float
        width of the transition region in unit of x
    power : float
        tweak the steepness of the curve
    Returns
    -------
    sigmoid : array
        sigmoid, same length of x"""
    return 1.0 / (1 + np.exp(-power * (x - x0 - width / 2) / width))

In [ ]:
import matplotlib.pyplot as plt

# Assuming you already have ell_HE and your sigmoid function defined
plt.figure(figsize=(8, 5))

# Plot the data
plt.plot(ell_HE, sigmoid(ell_HE, ell_fit_high, ell_fit_high / 10), label=r'$\mathrm{Sigmoid\ Fit}$', color='darkviolet', linewidth=2)
 
# Add labels
plt.xlabel(r'Multipole $\ell$', fontsize=14)
plt.ylabel('Sigmoid', fontsize=14)

# Add title
plt.title('Sigmoid Function', fontsize=16)

# Add legend
plt.legend(fontsize=12)

# Add grid
plt.grid(True, which='both', linestyle='--', alpha=0.6)

# Improve tick font size
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Tight layout
plt.tight_layout()

# Show the plot
plt.show()

We use the extrapolated power law to generate small-scale realizations. To isolate the small-scale modes in the extrapolated $C_{\ell}$, we apply a sigmoid function as a smooth high pass filter.

In [ ]:
ell_final=np.arange(lmax_final+1)
cl_norm_final= ((ell_final * (ell_final + 1)) / (2*np.pi))
cl_norm_final[0] = 1

In [ ]:
powerlawscaling=np.zeros(lmax_final+1)
powerlawscaling[2:] = powerlaw_model(ell_final[2:], A_fit2, gamma_fit2)
powerlawscaling[:2] = 0

# Underlying (unfiltered) power-law C_ell in logspace
cl_pl = powerlawscaling / cl_norm_final

# Transition weight and harmonic-space filters
w_final = sigmoid(ell_final, ell_fit_high, ell_fit_high / 10)
f_ss_final = np.sqrt(w_final)

# Filtered small-scale C_ell (for reference/plots): filter^2 = w_final
cl_ss = cl_pl * w_final


In [ ]:
import matplotlib.pyplot as plt

# Assuming you already have ell_HE and your sigmoid function defined
plt.figure(figsize=(8, 5))

# Plot the data
plt.plot(ell_final, cl_ss*cl_norm_final, label='Filtered Power-law (Only small scales)', color='orangered', linewidth=2)
plt.plot(ell_final, powerlawscaling, label='Power-law', color='darkgreen', ls='--')

# Add labels
plt.xlabel(r'Multipole $\ell$', fontsize=14)
plt.ylabel(r"$\ell(\ell+1)C_\ell/2\pi\;$", fontsize=14)

# Add title
plt.title('Fitted Power Spectrum', fontsize=16)

# Add legend
plt.legend(fontsize=12)

# Add grid
plt.grid(True, which='both', linestyle='--', alpha=0.6)

# Improve tick font size
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.xscale('log')
plt.yscale('log')

plt.xlim([30,3*lmax_final])

# Tight layout
plt.tight_layout()

# Show the plot
plt.show()

## Preparing the Modulation Map for Intensity

In [ ]:
import pymaster as nmt

#### Prepare the mask for computing the power spectrum of ff for modulation map

In [ ]:
maskmod=np.zeros_like(log_he_ff)
maskmod[log_he_ff>4]=1.0
# hp.mollview(log_he_ff*maskmod,cmap='turbo',norm='hist',title='Masked Map (Masking threshold=4 K)',unit=r'${\rm T\,[\mu K}_{\rm RJ}]$',min=1,max=8.0)

In [ ]:
log_he_ff2=hp.ud_grade(log_he_ff,nside_out=128)  #Downgrade to nside=128

In [ ]:
maskmod = hp.ud_grade(maskmod, nside_out=hp.get_nside(log_he_ff2))  #Downgrade mask to nside=128
lmaxmod=100                                             # lmax for computing the Cl
apo_mask = nmt.mask_apodization(maskmod, 5, apotype="C2")       #apodization of the mask 
ell0, norm, cl0,dl0 = run_anafast(log_he_ff2*maskmod , lmax=lmaxmod)  # compute the power spectrum of the downgraded map (after masking)


In [ ]:
nsidepatches = 8    #nside=8 used for preparing the mod map
centers = np.vstack(hp.pix2vec(ipix=np.arange(hp.nside2npix(nsidepatches)), nside=nsidepatches)).T  #3d (x,yz) coordinates of the pixels with nside=8. 

In [ ]:
fit_model = lambda x, Ad, alpha: Ad * (x / 100) ** (alpha)

tmod = np.zeros_like(log_he_ff2)
n_eff = np.zeros_like(log_he_ff2)


def bin_cell(cell, dig):
    cb = []
    errb = []
    for i in np.unique(dig):
        msk = dig == i
        cb.append(cell[msk].mean())
        errb.append(cell[msk].std())

    return np.array(cb), np.array(errb)


def bin_ell(ells, dig):
    lb = []
    dl = []
    for i in np.unique(dig):
        msk = dig == i
        lb.append(ells[msk].mean())
        dl.append((ells[msk].max() - ells[msk].min()) / 2)
    return np.array(lb), np.array(dl)

paramss1=[]
paramss2=[]

print("Total number of patches to be analyzed: ", len(centers))

randpatches=[]
for ipix, c in enumerate(centers):
    patch = np.zeros_like(log_he_ff2)    #we are making a circular patch for every pixel in the nside=8 map
    maskpixs = hp.query_disc(nside=hp.get_nside(log_he_ff2), vec=c, radius=np.radians(7.6))
    patch[maskpixs] = 1

    apo_patch = nmt.mask_apodization(patch, 5, apotype="C2")  #Apodization of the patch 
    fsky = apo_patch.sum() / apo_patch.size    



    ellp, norm, clp,dlp = run_anafast(m=log_he_ff2 * apo_patch, lmax=lmaxmod)   #compute the power spectrum of the data (downgraded at nside=128) in the patch region
    digi = np.digitize(ellp, np.linspace(0, lmaxmod, 10))   #put each ellp values in a bin out of 10 bins (bewtwen 0 and lmaxmod); lmaxmaxmod=100

    dtt, errtt = bin_cell((clp), digi) / fsky   #compute the mean and std of the power spectrum in each bin

    lb, delta_l = bin_ell(ellp, digi)   #get the ell for each bin and the delta_l (width of the bin)

    lmaskt = np.logical_and((lb) < 100, lb > 50)



    




    param_tt, _ = curve_fit(
        fit_model, ydata=dtt[lmaskt], xdata=lb[lmaskt], sigma=errtt[lmaskt]
    )


    
    #print(param_tt)

    l_ = 80  #Try with 50 or 60. 
    tmod += np.sqrt(fit_model(l_, *param_tt) / cl0[l_]) * apo_patch   #compute the mod map for each patch (mod map is at nside=128), it's the sum of tmod for each patch. 

    
    if ipix % 100 == 0:
        print(ipix)


    n_eff += apo_patch



In [ ]:
tsm_raw = np.divide(tmod, n_eff, out=np.zeros_like(tmod), where=n_eff > 0)
tsm = hp.smoothing(tsm_raw, fwhm=np.radians(5.5))
hp.mollview(tsm, title=r"Modulation Map (With Radius of Patch=$7.6^{\circ}$)",cmap='turbo',norm='hist')
hp.graticule(dpar=30)

### Filtering the Large-scale map (using sigmoid filter as low pass filter)

In [ ]:
np.random.seed(777)
# filter large scales
alm_ff_fullsky = hp.map2alm(log_he_ff, lmax=lmax, use_pixel_weights=True)  
ii_LS_alm = np.empty_like(alm_ff_fullsky)

In [ ]:
# transition weight (computed at input-map lmax)
w_he = sigmoid(ell_HE, x0=ell_fit_high, width=ell_fit_high / 10)
f_ls_he = np.sqrt(1.0 - w_he)
ii_LS_alm = hp.almxfl(alm_ff_fullsky, f_ls_he)


In [ ]:
plt.loglog(hp.alm2cl(alm_ff_fullsky),color='gold',label='fullsky')
plt.loglog(hp.alm2cl(ii_LS_alm),color='teal',label='sig filtered')

plt.grid()
plt.axvline(
    ell_fit_high,
    color="grey",
    linestyle="--",
    label=rf"$\ell={ell_fit_high}$")

plt.xlabel(r'Multipole $\ell$', fontsize=14)
plt.ylabel(r"$C_\ell$", fontsize=14);


plt.legend()

## Adding small-scale map to large-scale map

In [ ]:
# Generating small-scale realizations using synfast (from underlying power law)
map_ss_full = hp.synfast(cl_pl, lmax=lmax_final, new=True, nside=nside_final)

# Keep only small-scale modes using the same transition filter used for the large-scale map
alm_ss_full = hp.map2alm(map_ss_full, lmax=lmax_final)
alm_ss = hp.almxfl(alm_ss_full, f_ss_final)
map_ss = hp.alm2map(alm_ss, nside=nside_final)

# Modulate small-scale map with the modulation map
map_ss_modulated = map_ss * hp.ud_grade(
    tsm / np.sqrt(np.mean(tsm**2)),
    nside_out=nside_final,
)

# Get the sigmoid-filtered large-scale map
map_ls = hp.alm2map(ii_LS_alm, nside=nside_final)

# Final addition: smallscale + large scale
ii_map_out = map_ss_modulated + map_ls


In [ ]:
plt.figure(figsize=(12, 8))  # make the canvas big enough

hp.mollview(log_he_ff,
        cmap="turbo",
        norm="hist",
        title='Original Free-Free Map (logspace) -- Nside=256',
        sub=(1, 2, 1),  # (nrows, ncols, index)
    )

hp.mollview(ii_map_out,
        cmap="turbo",
        norm="hist",
        title=f'Free-Free Map with small-scales injected (logspace) -- Nside={nside_final}',
        sub=(1, 2, 2),  # (nrows, ncols, index)
    )

##### 2D plot for maps at different stages of small-scale addition

In [ ]:
plt.figure(figsize=(15, 5))
hp.gnomview(
    ii_map_out,
    reso=3.75,
    xsize=320,
    rot=[30, -58],
    sub=155,cmap='turbo',norm='hist',
    title="small-scales added i ",
)
hp.gnomview(
    map_ls,
    reso=3.75,
    xsize=320,
    rot=[30, -58],
    sub=154,cmap='turbo',norm='hist',
    title="large-scales i  ",
)
hp.gnomview(
    map_ss, reso=3.75, xsize=320, rot=[30, -58], sub=151,cmap='turbo', title="small-scales i",norm='hist'
)
hp.gnomview(map_ss_modulated, reso=3.75, xsize=320, rot=[30, -58], sub=153,cmap='turbo', title="small-scales modulated  i ",norm='hist')
hp.gnomview(tsm, reso=3.75, xsize=320, rot=[30, -58], sub=152,cmap='turbo', title="modulation i ",norm='hist')



lon,lat=130,48
plt.figure(figsize=(15, 5))
hp.gnomview(
    ii_map_out,
    reso=3.75,
    xsize=320,
    rot=[lon, lat],norm='hist',
    sub=155,cmap='turbo',
    title="small-scales added i ",
)
hp.gnomview(
    map_ls,
    reso=3.75,
    xsize=320,
    rot=[lon, lat],norm='hist',
    sub=154,cmap='turbo',
    title="large-scales  i  ",
)
hp.gnomview(
    map_ss, reso=3.75, xsize=320, 
    rot=[lon, lat], sub=151,
    cmap='turbo',norm='hist',
    title="small-scales i"
)
hp.gnomview(map_ss_modulated, reso=3.75, xsize=320,
             rot=[lon, lat], sub=153,cmap='turbo',norm='hist',
               title="small-scales modulated  i ")
hp.gnomview(tsm, reso=3.75, xsize=320,
             rot=[lon, lat], sub=152,cmap='turbo',norm='hist',
               title="modulation i ")



#### Power Spectra of maps at different stages

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(10, 7))  

# Input Cl for the small-scale map
plt.loglog(cl_ss * cl_norm_final, color='green', label='small scale Cl (fitted)')

# small-scale Cl
ell_map_ss, cl_norm_map_ss, cl_map_ss, dl_map_ss = run_anafast(map_ss, lmax_final)
plt.loglog(ell_map_ss, cl_map_ss * cl_norm_map_ss, color='orangered', label='small scale Cl (synfast-generated)')

# modulated small-scale Cl
ell_map_ss_mod, cl_norm_map_ss_mod, cl_map_ss_mod, dl_map_ss_mod = run_anafast(map_ss_modulated, lmax_final)
plt.loglog(ell_map_ss_mod, cl_map_ss_mod * cl_norm_map_ss_mod, color='blue', label='small scale Cl (modulated)')

# large-scale Cl
plt.loglog(((ell_HE * (ell_HE + 1)) / (2*np.pi))*hp.alm2cl(ii_LS_alm), color='magenta', label='large-scale Cl (sigmoid-filtered)')

# total map Cl
ell_map_tot, cl_norm_map_tot, cl_map_tot, dl_map_tot = run_anafast(ii_map_out, lmax_final)
plt.loglog(ell_map_tot, cl_map_tot * cl_norm_map_tot, color='gold', label='Output Cl')

# Cl for the original HE map
plt.loglog(ell_HE, ((ell_HE * (ell_HE + 1)) / (2*np.pi))*cl_HE,color='black',label='HE - Input',ls='-')


plt.legend()
plt.xlabel(r'$\ell$')
plt.ylabel(r'$C_\ell$')
plt.title('Angular Power Spectra Comparison')


plt.tight_layout()


# plt.savefig('cls_final.png', dpi=300)
plt.show()

#### Power spectrum comparison of input vs small-scale added maps (logspace)

In [ ]:
ell, cl_norm, cltot_final,dltot_final = run_anafast(ii_map_out, lmax_final)

In [ ]:
plt.figure(figsize=(10, 5))

plt.loglog(ell, ell * (ell + 1) / np.pi / 2 * cltot_final, label=r"$C_\ell$ (small-scales added)")
plt.loglog(ell_HE, ((ell_HE * (ell_HE + 1)) / (2*np.pi)) * cl_HE,
           color='limegreen', label=r'HE - Input')

plt.axvline(
    ell_fit_high,
    linestyle="--",
    color="gray",
    label=rf"$\ell={ell_fit_high}$"
)

plt.axvline(
    ell_fit_low,
    linestyle="--",
    color="gray",
    label=rf"$\ell={ell_fit_low}$"
)

plt.legend()
plt.grid()
plt.title(
    rf"Temp spectrum for free-free Log. Tens  $\gamma_{{fit}}={gamma_fit2:.2f}$"
)
plt.ylabel(r"$\ell(\ell+1)C_\ell/2\pi\ [\mu K_{RJ}^2]$")
plt.xlabel(r"$\ell$")
# plt.ylim(0.7e-2,0.7e-1)
# plt.xlim(130,170)
#plt.xlim(2, lmax_final+1000)

### LogSpace --> LinearSpace Transformation

In [ ]:
output_map = log_pol_tens_to_map(ii_map_out)
hp.mollview(output_map, title="Final Free-Free Intensity Map at 30 GHz (with small-scales)", cmap='turbo', unit=r'${\rm T\,[\mu K}_{\rm RJ}]$',norm='hist')

#### Comparison of the input vs small-scale added maps (linearspace; nside_inp=256, nside_out=nside_final)

In [ ]:
plt.figure(figsize=(12, 8))  # make the canvas big enough

hp.mollview(HE_ff_T,
        cmap="turbo",
        norm="hist",
        title='Original Free-Free Map (I) (Nside=256',
        sub=(1, 2, 1),  # (nrows, ncols, index)
        unit=r'${\rm T\,[\mu K}_{\rm RJ}]$',
    )

hp.mollview(output_map,
        cmap="turbo",
        norm="hist",
        title=f'Free-Free Map with small-scales injected (I) (Nside={nside_final})',
        sub=(1, 2, 2),  # (nrows, ncols, index)
        unit=r'${\rm T\,[\mu K}_{\rm RJ}]$',
    )

### Maplevel comparison at different stages

In [ ]:
lat = 15
plt.figure(figsize=(15, 10))

hp.gnomview(
    HE_ff_T,
    title="I (no small scales) ",
    rot=[0, lat],
    reso=3.75,
    xsize=320,norm='hist',
    sub=231,cmap='turbo',
)

hp.gnomview(
    log_he_ff,
    title="i (no small scales) ",
    rot=[0, lat],
    reso=3.75,
    xsize=320,norm='hist',
    sub=232,cmap='turbo',
)

hp.gnomview(
    (map_ss),
    title="small scales ",
    rot=[0, lat],
    reso=3.75,
    xsize=320,norm='hist',
    sub=233,cmap='turbo',
)

hp.gnomview(
    (map_ss_modulated),
    title="small scales (modulated) ",
    rot=[0, lat],
    reso=3.75,
    xsize=320,norm='hist',
    sub=234,cmap='turbo',
)

hp.gnomview(
    ii_map_out,
    title="i w/ small scales ",
    rot=[0, lat],
    reso=3.75,norm='hist',
    xsize=320,
    sub=235,cmap='turbo',
)


hp.gnomview(
    output_map,
    title="I w/ small scales ",
    rot=[0, lat],
    reso=3.75,norm='hist',
    xsize=320,
    sub=236,cmap='turbo',
)

In [ ]:
lat = 15
lon=65

plt.figure(figsize=(15, 10))

hp.gnomview(
    HE_ff_T,
    title="I (no small scales) ",
    rot=[lon, lat],
    reso=3.75,
    xsize=320,norm='hist',
    sub=231,cmap='turbo',
)

hp.gnomview(
    log_he_ff,
    title="i (no small scales) ",
    rot=[lon, lat],
    reso=3.75,
    xsize=320,norm='hist',
    sub=232,cmap='turbo',
)

hp.gnomview(
    (map_ss),
    title="small scales ",
    rot=[lon, lat],
    reso=3.75,norm='hist',
    xsize=320,
    sub=233,cmap='turbo',
)

hp.gnomview(
    (map_ss_modulated),
    title="small scales (modulated) ",
    rot=[lon, lat],
    reso=3.75,
    xsize=320,norm='hist',
    sub=234,cmap='turbo',
)

hp.gnomview(
    ii_map_out,
    title="i w/ small scales ",
    rot=[lon, lat],
    reso=3.75,
    xsize=320,norm='hist',
    sub=235,cmap='turbo',
)


hp.gnomview(
    output_map,
    title="I w/ small scales ",
    rot=[lon, lat],
    reso=3.75,
    xsize=320,norm='hist',
    sub=236,cmap='turbo',
)

### Comparing the Cl for the input and output HE ff maps

In [ ]:
ell, cl_norm, cltotout,dltotout = run_anafast(output_map, lmax_final)
ellI, cl_normI, cltotI,dltotI = run_anafast(HE_ff_T, lmax)


In [ ]:
plt.figure(figsize=(10, 5))

plt.loglog(ellI, ellI * (ellI + 1) / np.pi / 2 * cltotI, label=r"HE  $C_\ell$ -- Input")
plt.loglog(ell, ell * (ell + 1) / np.pi / 2 * cltotout, label=r"tot  $C_\ell$ -- Output")

plt.axvline(
    ell_fit_high,
    linestyle="--",
    color="gray",
    label=rf"$\ell={ell_fit_high}$"
)

plt.axvline(
    ell_fit_low,
    linestyle="--",
    color="gray",
    label=rf"$\ell={ell_fit_low}$"
)

plt.legend()
plt.grid()

plt.ylabel(r"$\ell(\ell+1)C_\ell/2\pi\ [\mu K_{RJ}^2]$")
plt.xlabel(r"$\ell$")
plt.xlim(2, lmax_final+1000)

### Saving the final map 

In [ ]:
# Output HE free-free map at 30 GHz (with small-scales) at desired nside

fname = DATA_DIR / f"ff_highres_map_nside{nside_final}_f2_30GHz.fits"
hp.write_map(
    fname,
    output_map,
    overwrite=True,
    dtype=np.float64,
    column_units="uK_RJ",
)
output_files.append(fname)

# Modulation map (dimensionless)

nside_mod = hp.get_nside(tsm)
fname = DATA_DIR / f"ff_modulation_map_nside{nside_mod}_f2_30GHz.fits"
hp.write_map(
    fname,
    tsm,
    overwrite=True,
    dtype=np.float64,
    column_units="dimensionless",
)
output_files.append(fname)

# Input HE free-free map at 30 GHz (without small-scales) at nside=256

fname = DATA_DIR / f"ff_HE_map_nside{nside}_30GHz.fits"
hp.write_map(
    fname,
    HE_ff_T,
    overwrite=True,
    dtype=np.float64,
    column_units="uK_RJ",
)
output_files.append(fname)